In [ ]:
# HTTP lib
import requests

# Ticker to CIK lookup
There are two lists referenced here. One is for companies and the other is funds. Respective urls: https://www.sec.gov/files/company_tickers.json, https://www.sec.gov/data/company_tickers_mf.json. We start with company_tickers.json and if unsuccessful try company_tickers_mf. CIK is padded to 10 digits with leading 0's.

In [ ]:
# Define method
def cik_from_ticker(target_ticker):
  target_cik = "0000000000"

  # Needed to interact with SEC website
  request_headers = { "User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"} 

  # First try company_tickers.json
  list_url = "https://www.sec.gov/files/company_tickers.json"
  response = requests.get(url = list_url, headers = request_headers)
  response.raise_for_status()

  json_data = response.json()

  # Keys are simply nonnegative integers starting at 0. Each company's entry is structured as following AAPL example: {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'}
  for key, value in json_data.items():
    if value['ticker'] == target_ticker:
      target_cik = value['cik_str']
      break

  # If CIK is still set to 0000000000, we know company_tickers.json failed. Try MF's
  if target_cik == "0000000000":
    list_url = "https://www.sec.gov/data/company_tickers_mf.json"
    response = requests.get(url = list_url, headers = request_headers)
    response.raise_for_status()

    json_data = response.json()

    # Loop through each entry in the "data" dict. Ticker is the 4th (0,1,2,3) element in each entry. CIK is 1st. Obviously, JSON response is different structure than company_tickers.json
    for current_entry in json_data['data']:
      if current_entry[3] == target_ticker:
        target_cik = current_entry[0]
        break

  # Pad to 10 digits with leading 0's
  return str(target_cik).zfill(10)

In [ ]:
# Example of calling above function:
test_target_ticker = "VUZI"
test_target_cik = cik_from_ticker(test_target_ticker)

# Verify CIK and print
if test_target_cik == "0000000000":
  print("Failed to obtain CIK for {}".format(test_target_ticker))
else:
  print("Ticker: {}, CIK: {}".format(test_target_ticker, test_target_cik))

Ticker: VUZI, CIK: 0001463972


# CIK to Entity Name and + Ticker
Just in case we want to quickly get an entity name and/or ticker from a CIK, for example after stripping it from an accession number, refer to the same 2 lists referenced above.

In [ ]:
def cik_to_name(cik_to_search):

  # Define return strings to begin
  entity_name = "*Unknown Name*"
  entity_ticker = "*Unknown Ticker*"

  # Pad given CIK to 10 digits. Also can accept either an int or string
  padded_to_search = str(cik_to_search).zfill(10)

  # UA
  request_headers = { "User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36" }

  # Again, try company list first
  list_url = "https://www.sec.gov/files/company_tickers.json"
  response = requests.get(url = list_url, headers = request_headers)
  response.raise_for_status()

  json_data = response.json()

  # Loop through. Again pad each as we compare
  for key, value in json_data.items():
    if str(value['cik_str']).zfill(10) == padded_to_search:
      entity_name = value['title']
      entity_ticker = "$" + value['ticker']
      break

  # If name or ticker is still unknown, give the mutual funds page a shot. Keep in mind there is no "name" given. Rather, a seriesID and classID are given along with a ticker.
  # https://www.sec.gov/open/datasets-investment_company gives information and download links to XML files which can be parsed to garner more information on the fund name and the investment company behind it
  if (entity_name == "*Unknown Name" or entity_ticker == "*Unkown Ticker"):

    list_url = "https://www.sec.gov/data/company_tickers_mf.json"
    response = requests.get(url = list_url, headers = request_headers)
    response.raise_for_status()

    json_data = response.json()

    # Loop through each entry in the "data" dict. CIK is first element
    for current_entry in json_data['data']:
      if str(current_entry[0]).zfill(10) == padded_to_search:

        # TODO: Implement a parsing of the data as given on https://www.sec.gov/open/datasets-investment_company to return information about the fund 
        entity_name = "*Mutual Fund*"
        entity_ticker = "$" + current_entry[3]
        break

  return entity_name + ", " + entity_ticker

In [ ]:
print(cik_to_name(1463972))